In [11]:
import pandas as pd
import os

#### Add local paths to cluster data

In [12]:
PRE = "/Users/damoncrockett/Desktop/cities_20k/"

In [13]:
DB = "/Users/damoncrockett/Desktop/cities_20k/yahoo/NEW_Clusters/Clusters_Normalized/Normalized_Cluster_Labels/"

In [14]:
data_files = []
for root,dirs,files in os.walk(DB):
    for name in files:
        data_files.append(os.path.join(root,name))

In [15]:
# takes a few minutes to run

for item in data_files:
    tmp = pd.read_csv(item)
    n = len(tmp.index)
    local_path = []
    for i in range(n):
        local_path.append(PRE+tmp.city.loc[i]+"_20k/"+tmp.image_id.loc[i]+".jpg")
    tmp['local_path'] = local_path
    tmp.to_csv(item,index=False)

In [16]:
from PIL import Image

In [17]:
import numpy as np

#### Now making square montage of every cluster (there should be 92)

In [18]:
thumb_side = 64

In [19]:
for item in data_files:
    category = os.path.basename(item).rstrip("csv").rstrip(".")
    folder = item.split("/")[9]
    df = pd.read_csv(item)
    bins = list(set(list(df.cluster_id)))
    
    for member in bins:
        outfile = PRE+"viz/normalized/"+category+"_"+folder+"_"+str(member)+".png"
        
        tmp = df[df.cluster_id==member]
        n = len(tmp.index)
        
        side = int(round(np.sqrt(n)) - 1)
        if side > 0:
            m = side * side
            tmp = tmp[:m]
            n = len(tmp.index)
            tmp.reset_index(drop=True,inplace=True)
        
            tmp["y_coord"] = list(np.repeat(range(side),side) * thumb_side)
            exes = list(range(side)) * side
            tmp["x_coord"] = [thumb_side * item for item in exes]
        
            canvas = Image.new('RGB',(side * thumb_side, side * thumb_side),(50,50,50))
            for i in range(n):
                thumb = Image.open(tmp.local_path.loc[i])
                thumb.thumbnail((thumb_side,thumb_side),Image.ANTIALIAS)
                canvas.paste(thumb,(tmp.x_coord.loc[i],tmp.y_coord.loc[i]))
    
            canvas.save(outfile)